In [12]:
import numpy as np
import pandas as pd
import openpyxl
import copy
# Get which order the user wants for the autoregressive model
order = int(input("Order for Autoregressive model"))

# create an array of shape(order, ) and input coefficients in it
coeff = np.zeros((order, ))
for idx in range(order) :
    coeff[idx] = float(input("coefficient : " + str(idx + 1)))
coeff.reshape((-1, 1))
# print(coeff)

# get the initial values of the time series
initial = np.zeros((order, ))
for idx in range(order):
    initial[idx] = float(input("initial value at time " + str(idx + 1) + " : "))
# print (initial)

# get number of time steps
len = int(input("Length of time series to be generated : "))

# write initial data in the excel file
wb = openpyxl.load_workbook("Data.xlsx")
ws = wb.active
for idx in range(order):
    ws.cell(row=1, column=idx+1).value = initial[idx]

# generate the autoregressive time series of the given order
# latest stores the latest values of the times series
latest = copy.deepcopy(initial)

for idx in range(len):
    # multiply the matrices
    val = np.matmul(latest, coeff) + np.random.normal(0, 1)
    np.roll(latest, -1)
    latest[order-1] = val
    ws.cell(row=1, column=order+idx+1).value = val
    print(val)

wb.save("Data.xlsx")

3.107592444382212
4.9559479137525795
9.364911005725618
19.07696820544015
37.07475249867846
71.6373702045639
144.74918716237855
288.75914419649644
578.4559351038039
1156.4405355753015
